# Pytorch Tutorial

Pytorch is a popular deep learning framework and it's easy to get started.

In [1]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import time

BATCH_SIZE = 128
NUM_EPOCHS = 10

First, we read the mnist data, preprocess them and encapsulate them into dataloader form.

In [2]:
# preprocessing
normalize = transforms.Normalize(mean=[.5], std=[.5])
transform = transforms.Compose([transforms.ToTensor(), normalize])

# download and load the data
train_dataset = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./mnist/', train=False, transform=transform, download=False)

# encapsulate them into dataloader form
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

Then, we define the model, object function and optimizer that we use to classify.

In [3]:
class SimpleNet(nn.Module):
# TODO:define model
    def __init__(self):
        super(SimpleNet, self).__init__()    # super用法:Cnn继承父类nn.Model的属性，并用父类的方法初始化这些属性
        self.conv = nn.Sequential(     #padding=2保证输入输出尺寸相同(参数依次是:输入深度，输出深度，ksize，步长，填充)
            nn.Conv2d(1, 6, 5, stride = 1, padding = 2),
            nn.ReLU(True),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(6, 16, 5),
            nn.ReLU(True), 
            nn.MaxPool2d(2, 2))

        self.fc = nn.Sequential(
            nn.Linear(400, 120), 
            nn.Linear(120, 84), 
            nn.Linear(84, 10))

    def forward(self, x):
        output = self.conv(x)
        output = output.view(BATCH_SIZE, -1)
        output = self.fc(output)
        return output
    
model = SimpleNet()
gpu = torch.cuda.is_available()
if gpu:
    model = model.cuda()
# TODO:define loss function and optimiter
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 1e-2)

Next, we can start to train and evaluate!

In [4]:
# train and evaluate
for epoch in range(NUM_EPOCHS):
    epoch_loss = 0.0
    epoch_acc = 0.0
    for images, labels in tqdm(train_loader):
        # TODO:forward + backward + optimize
        if gpu:
            images = images.cuda()
            labels = labels.cuda()
        output = model(images)
        loss = criterion(output, labels)
        epoch_loss += loss.item() * labels.size(0)
        _, prediction = torch.max(output, 1)
        num_c = (prediction == labels).float().sum()
        acc = (prediction == labels).float().mean()
        epoch_acc += num_c.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    '''print('{} epoch, loss: {:.6f}, acc: {:.6f}'.format(
        epoch + 1, epoch_loss / (len(train_dataset)), epoch_acc / len(train_dataset)))'''
        
    # evaluate
    # TODO:calculate the accuracy using traning and testing dataset

eval_loss = 0
eval_acc = 0
for images, labels in train_loader:
    if gpu:
        images = images.cuda()
        labels = labels.cuda()
    output = model(images)
    loss = criterion(output, labels)
    eval_loss += loss.item() * labels.size(0)
    _, prediction = torch.max(output, 1)
    num_c = (prediction == labels).sum()
    eval_acc += num_c.item()
# print('Train loss: {:.6f}, acc: {:.6f}'.format(eval_loss / (len(train_dataset)), eval_acc / (len(train_dataset))))
print('Train acc: {:.6f}'.format(eval_acc / (len(train_dataset)))) # Train acc: 0.975783
eval_loss = 0
eval_acc = 0
for images, labels in test_loader:
    if gpu:
        images = images.cuda()
        labels = labels.cuda()
    output = model(images)
    loss = criterion(output, labels)
    eval_loss += loss.item() * labels.size(0)
    _, prediction = torch.max(output, 1)
    num_c = (prediction == labels).float().sum()
    eval_acc += num_c.item()
# print('Test loss: {:.6f}, acc: {:.6f}'.format(eval_loss / (len(test_dataset)), eval_acc / (len(test_dataset))))  
print('Test acc: {:.6f}'.format(eval_acc / (len(test_dataset)))) # Test acc: 0.977800



100%|████████████████████████████████████████████████████████████████████████████████| 468/468 [00:13<00:00, 34.42it/s]


Train acc: 0.975783
Test acc: 0.977800


#### Q5:
Please print the training and testing accuracy.